In [1]:
from os import path, environ
from itertools import filterfalse, repeat, chain
from functools import partial
from operator import itemgetter, contains
from collections import namedtuple
from six import string_types
import urllib.parse

from IPython.display import Image, display, HTML

import numpy as np
import pandas as pd

df = pd \
       .read_excel('/'.join(('file://localhost',
                             path.expanduser('~').replace(path.sep, '/'),
                             'OneDrive - The University of Sydney (Students)',
                             'DR SPOC - Graders 1 and 2.xlsx')),
                   skiprows=1, header=[0,1], index_col=[0])
#\
#       .set_index([None, 'Folder Name'])
#       .rename(index={'Name': 'fooooo'})

# Made no difference:
#df.dropna(subset=tuple(column
#                       for column in df.columns
#                       if 'disc-centred photo' in column),
#          how='all', inplace=True)

display(HTML('<h2>Filtered DR SPOC - Graders 1 and 2</h2>'))

#df.axes[1].names = pd.core.indexes.frozen.FrozenList(
#    chain.from_iterable(repeat(tuple(filter(None, df.axes[1].names)),
#                               len(df.axes[1].names)))
#)
df

R1 (Right macula-centred image)                \
Folder Name Overall quality of the photographs taken ETDRS Grading   
27                                               2.0           0.0   
50                                               1.0           2.0   
73                                               5.0           8.0   
118                                              5.0           8.0   
145                                              2.0           0.0   
...                                              ...           ...   
9800                                             3.0           0.0   
9819                                             5.0           8.0   
9897                                             1.0           0.0   
9938                                             2.0           2.0   
9998                                             1.0           2.0   

                             R2 (Right optic-disc centred image)  \
Folder Name Maculopathy Overall quality of the photographs taken   
27                  0.0                                      1.0   
50                  0.0                                      2.0   
73                  5.0                                      NaN   
118                 5.0                                      5.0   
145                 0.0                                      3.0   
...                 ...                                      ...   
9800                5.0                                      5.0   
9819                5.0                                      5.0   
9897                0.0                                      1.0   
9938                1.0                                      2.0   
9998                2.0                                      2.0   

                                       \
Folder Name ETDRS Grading Maculopathy   
27                    0.0         0.0   
50                    1.0         0.0   
73                    NaN         NaN   
118                   8.0         5.0   
145                   0.0         0.0   
...                   ...         ...   
9800                  8.0         5.0   
9819                  8.0         5.0   
9897                  0.0         0.0   
9938                  2.0         1.0   
9998                  0.0         0.0   

                      L1 (Left macula-centred image)                \
Folder Name Overall quality of the photographs taken ETDRS Grading   
27                                               1.0           0.0   
50                                               1.0           0.0   
73                                               5.0           8.0   
118                                              5.0           8.0   
145                                              2.0           0.0   
...                                              ...           ...   
9800                                             5.0           8.0   
9819                                             5.0           8.0   
9897                                             1.0           0.0   
9938                                             2.0           2.0   
9998                                             3.0           0.0   

                              L2 (Left optic-disc centred image)  \
Folder Name Maculopathy Overall quality of the photographs taken   
27                  0.0                                      1.0   
50                  0.0                                      1.0   
73                  5.0                                      NaN   
118                 5.0                                      5.0   
145                 0.0                                      1.0   
...                 ...                                      ...   
9800                5.0                                      5.0   
9819                5.0                                      5.0   
9897                0.0                                      1.0   
9938                1.0         

In [2]:
cols = df.axes[1]
conv = dict(zip(cols ,[str] * len(cols)))
conv

{('R1 (Right macula-centred image)',
  'Overall quality of the photographs taken'): str,
 ('R1 (Right macula-centred image)', 'ETDRS Grading'): str,
 ('R1 (Right macula-centred image)', 'Maculopathy'): str,
 ('R2 (Right optic-disc centred image)',
  'Overall quality of the photographs taken'): str,
 ('R2 (Right optic-disc centred image)', 'ETDRS Grading'): str,
 ('R2 (Right optic-disc centred image)', 'Maculopathy'): str,
 ('L1 (Left macula-centred image)',
  'Overall quality of the photographs taken'): str,
 ('L1 (Left macula-centred image)', 'ETDRS Grading'): str,
 ('L1 (Left macula-centred image)', 'Maculopathy'): str,
 ('L2 (Left optic-disc centred image)',
  'Overall quality of the photographs taken'): str,
 ('L2 (Left optic-disc centred image)', 'ETDRS Grading'): str,
 ('L2 (Left optic-disc centred image)', 'Maculopathy'): str,
 ('Overall Finding', 'Overall Finding'): str}

In [3]:
column_list = []
for i in df.columns:
    column_list.append(i)
converter = {col: str for col in column_list} 
column_list

[('R1 (Right macula-centred image)',
  'Overall quality of the photographs taken'),
 ('R1 (Right macula-centred image)', 'ETDRS Grading'),
 ('R1 (Right macula-centred image)', 'Maculopathy'),
 ('R2 (Right optic-disc centred image)',
  'Overall quality of the photographs taken'),
 ('R2 (Right optic-disc centred image)', 'ETDRS Grading'),
 ('R2 (Right optic-disc centred image)', 'Maculopathy'),
 ('L1 (Left macula-centred image)',
  'Overall quality of the photographs taken'),
 ('L1 (Left macula-centred image)', 'ETDRS Grading'),
 ('L1 (Left macula-centred image)', 'Maculopathy'),
 ('L2 (Left optic-disc centred image)',
  'Overall quality of the photographs taken'),
 ('L2 (Left optic-disc centred image)', 'ETDRS Grading'),
 ('L2 (Left optic-disc centred image)', 'Maculopathy'),
 ('Overall Finding', 'Overall Finding')]

In [4]:
def chain_unique(*args):
    seen = set()
    yield from (v for v in chain(*args)
                if v not in seen and not seen.add(v))

display(HTML('<h2>Columns</h2>'))

display(HTML(
    '<ul>\n{}\n</ul>'.format('\n'.join(
        '  <li>"{}"</li>'.format(col)
        for col in chain_unique(map(itemgetter(1), df.axes[1]))
))))

In [5]:
manycat2threecat = {
    'Maculopathy': (
        'non-referable',    # [0] No diabetic maculopathy
        'referable',        # [1] HEx distant from the fovea
        'referable',        # [2] HEx approaching the fovea
        'referable',        # [3] HEx involving the fovea
        'referable',        # [4] Maculopathy, unspecified
        'No gradable image' # [5] No gradable image
    ),
    'ETDRS Grading': (        
        'non-referable',    # [0] No DR
        'referable',        # [1] Mild non-proliferative (mild pre-proliferative)
        'referable',        # [2] Moderate non-proliferative/ moderate pre-proliferative
        'referable',        # [3] Severe non-proliferative/ severe pre-proliferative
        'referable',        # [4] Proliferative retinopathy
        'referable',        # [5] Pre-retinal fibrosis+/- tractional retinal detachment
        'referable',        # [6] Treated proliferative retinopathy, Unstable
        'referable',        # [7] Treated proliferative retinopathy, Stable
        'No gradable image' # [8] No gradable image
    ),
    'Overall Findings': (
        np.nan,
        'referable',        # [1] Vision-threatening retinopathy
        'referable',        # [2] Non-proliferative diabetic retinopathy
        'non-referable',    # [3] No DR
        'No gradable image' # [4] Ungradable
    ),
    'Overall Quality of the Photographs Taken': (
        np.nan,
        'No gradable image', # [1] Inadequate for any diagnostic purpose
        'No gradable image', # [2] Unable to exclude emergent findings
        'No gradable image', # [3] Only able to exclude emergent findings
        'No gradable image', # [4] Not ideal but still able to exclude subtle findings
        'referable',         # [5] Ideal quality
    )
}

axes = filter(lambda c: c[:2] in frozenset(('R1', 'R2', 'L1', 'L2')),
              map(itemgetter(0), df.axes[1]))
columns = filterfalse(
    partial(contains,
            frozenset(
                ('Overall quality of the photographs taken',
                 'Overall Finding'))),
    chain_unique(map(itemgetter(1), df.axes[1])))

def to_manycat_name(o):
    if isinstance(o, string_types):
        o = o,
    
    for e in o[::-1]:
        lower_e = e.lower()
        if lower_e == 'overall quality of the photographs taken':
            return 'Overall Quality of the Photographs Taken'
        elif e.startswith('ETDRS') or e == 'Overall Findings':
            return e
        elif 'macul' in lower_e:
            # print('matched with: {!r}'.format(e))
            return 'Maculopathy'
        elif e.startswith('Overall Finding'):
            return 'Overall Findings'
        else:
            print('no match found for: {!r}'.format(e))
    
    raise TypeError('{!r} no key found for'.format(o))

def grad_mac2(series):
    def from_s(value):
        if pd.isnull(value) or isinstance(value, string_types):
            return value
        value = np.ushort(value)
        name = series.name if series.name in manycat2threecat else to_manycat_name(series.name)

        mapped = manycat2threecat.get(name)

        return value if mapped is None or len(mapped) < value else mapped[value]

    return series if series is None else series.apply(from_s)

df = df.transform(grad_mac2)
df

R1 (Right macula-centred image)                     \
Folder Name Overall quality of the photographs taken      ETDRS Grading   
27                                 No gradable image      non-referable   
50                                 No gradable image          referable   
73                                         referable  No gradable image   
118                                        referable  No gradable image   
145                                No gradable image      non-referable   
...                                              ...                ...   
9800                               No gradable image      non-referable   
9819                                       referable  No gradable image   
9897                               No gradable image      non-referable   
9938                               No gradable image          referable   
9998                               No gradable image          referable   

                                    R2 (Right optic-disc centred image)  \
Folder Name        Maculopathy Overall quality of the photographs taken   
27               non-referable                        No gradable image   
50               non-referable                        No gradable image   
73           No gradable image                                      NaN   
118          No gradable image                                referable   
145              non-referable                        No gradable image   
...                        ...                                      ...   
9800         No gradable image                                referable   
9819         No gradable image                                referable   
9897             non-referable                        No gradable image   
9938                 referable                        No gradable image   
9998                 referable                        No gradable image   

                                                   \
Folder Name      ETDRS Grading        Maculopathy   
27               non-referable      non-referable   
50                   referable      non-referable   
73                         NaN                NaN   
118          No gradable image  No gradable image   
145              non-referable      non-referable   
...                        ...                ...   
9800         No gradable image  No gradable image   
9819         No gradable image  No gradable image   
9897             non-referable      non-referable   
9938                 referable          referable   
9998             non-referable      non-referable   

                      L1 (Left macula-centred image)                     \
Folder Name Overall quality of the photographs taken      ETDRS Grading   
27                                 No gradable image      non-referable   
50                                 No gradable image      non-referable   
73                                         referable  No gradable image   
118                                        referable  No gradable image   
145                                No gradable image      non-referable   
...                                              ...                ...   
9800                                       referable  No gradable image   
9819                                       referable  No gradable image   
9897                               No gradable image      non-referable   
9938                               No gradable image          referable   
9998                               No gradable image      non-referable   

                                     L2 (Left optic-disc centred image)  \
Folder Name        Maculopathy Overall quality of the photographs taken   
27               non-referable                        No gradable image   
50               non-referable                        No gradable image   
73           No gradable image                                      NaN   
118          No gradable ima

In [6]:
print('- ', '\n- '.join(
    filter(lambda c: c[:2] in frozenset(('R1', 'R2', 'L1', 'L2')),
           map(itemgetter(0), df.axes[1]))
), sep='')

- R1 (Right macula-centred image)
- R1 (Right macula-centred image)
- R1 (Right macula-centred image)
- R2 (Right optic-disc centred image)
- R2 (Right optic-disc centred image)
- R2 (Right optic-disc centred image)
- L1 (Left macula-centred image)
- L1 (Left macula-centred image)
- L1 (Left macula-centred image)
- L2 (Left optic-disc centred image)
- L2 (Left optic-disc centred image)
- L2 (Left optic-disc centred image)


In [7]:
from operator import contains

print('- ', '\n- '.join(
    filterfalse(partial(contains, frozenset(
        ('Overall quality of the photographs taken',
         'Overall Finding'
        ))),
                chain_unique(map(itemgetter(1), df.axes[1])))
), sep='')

- ETDRS Grading
- Maculopathy


In [8]:
df.loc[145]

                                     Folder Name                             
R1 (Right macula-centred image)      Overall quality of the photographs taken    No gradable image
                                     ETDRS Grading                                   non-referable
                                     Maculopathy                                     non-referable
R2 (Right optic-disc centred image)  Overall quality of the photographs taken    No gradable image
                                     ETDRS Grading                                   non-referable
                                     Maculopathy                                     non-referable
L1 (Left macula-centred image)       Overall quality of the photographs taken    No gradable image
                                     ETDRS Grading                                   non-referable
                                     Maculopathy                                     non-referable
L2 (Left optic-disc centred ima

In [9]:
df.axes

[Int64Index([  27,   50,   73,  118,  145,  166,  197,  198,  202,  212,
             ...
             9644, 9648, 9689, 9716, 9784, 9800, 9819, 9897, 9938, 9998],
            dtype='int64', length=406),
 MultiIndex([(    'R1 (Right macula-centred image)', ...),
             (    'R1 (Right macula-centred image)', ...),
             (    'R1 (Right macula-centred image)', ...),
             ('R2 (Right optic-disc centred image)', ...),
             ('R2 (Right optic-disc centred image)', ...),
             ('R2 (Right optic-disc centred image)', ...),
             (     'L1 (Left macula-centred image)', ...),
             (     'L1 (Left macula-centred image)', ...),
             (     'L1 (Left macula-centred image)', ...),
             ( 'L2 (Left optic-disc centred image)', ...),
             ( 'L2 (Left optic-disc centred image)', ...),
             ( 'L2 (Left optic-disc centred image)', ...),
             (                    'Overall Finding', ...)],
            names=[None, 'Fo

In [10]:
display(HTML('<h2>Disc-centred photo counts</h2>'))
df[filter(lambda column: 'disc-centred photo' in column,
          df.columns)] \
  .apply(pd.value_counts)

27
50
73
118
145
...
9800
9819
9897
9938
9998


In [11]:
df[['What type of diabetes does the patient have?',
    'Who has performed the fundus grading in the clinical record?',
    'Diagnosis assessment by ophthalmologist?']] \
  .apply(pd.value_counts)

KeyError: "['What type of diabetes does the patient have?'\n 'Who has performed the fundus grading in the clinical record?'\n 'Diagnosis assessment by ophthalmologist?'] not in index"

In [ ]:
display(HTML('<h2>Reviewer</h2>'))
df[['Who has reviewed the fundus grading in the clinical record?']] \
  .apply(pd.value_counts)

In [ ]:
display(HTML('<h2>Eye history</h2>'))
df[filter(lambda column: column.startswith('Eye history'), df.columns)] \
  .apply(pd.value_counts)

In [ ]:
display(HTML('<h2>Photo grading</h2>'))
df[filter(lambda column: column.startswith('Photo grading'), df.columns)] \
  .apply(pd.value_counts)

In [ ]:
display(HTML('<h2>Ophthal photo grading</h2>'))
df[filter(lambda column: column.startswith('Ophthal photo grading'), df.columns)] \
  .apply(pd.value_counts)

In [ ]:
display(HTML('<h2>Columns</h2>'))

df = df[
    filterfalse(
        lambda col: any((col.startswith('Eye'),
                         #not col.startswith('Ophthal photo') and
                         col.startswith('Comorbidities'),
                         #not col.startswith('Photo grading') and
                         col.startswith('Complete'),
                         'OCT' in col,
                         'Visual Acuity' in col,
                         'IOP' in col,
                         'Timestamp' in col,
                         #'recorded' in col
                        )),
        df.columns)]

display(HTML(
    '<ul>\n{}\n</ul>'.format('\n'.join(
        '  <li>"{}"</li>'.format(col)
        for col in sorted(df.columns)
))))

In [ ]:
df[filter(lambda column: 'ICDRSS' in column or 'maculopathy' in column.lower(), df.columns)] \
  .apply(pd.value_counts)

- no DR & no diabetic maculopathy would classify as 'healthy'
- no gradable image would classify as 'ungradable' and 
- everything else would be 'DR'

In [ ]:
df['Photo grading - diabetes: Maculopathy recorded for the right eye'].unique()

In [ ]:
df[filter(lambda column: 'ICDRSS' in column, df.columns)] \
  .apply(pd.value_counts)

In [ ]:
df[filter(lambda column: column.endswith('eye maculopathy'), df.columns)] \
  .apply(pd.value_counts)

In [ ]:
df[filter(lambda column: 'hoto' in column, df.columns)].columns

In [ ]:
def upsert(obj, column, value):
    def _upsert():
        is_list = isinstance(obj.columns[column], list)
        if isinstance(obj.columns[column], tuple):
            obj.columns[column] = list(obj.columns[column]) + [value]
        elif len(obj.columns[column]) == 0:
            obj.columns[column] = [value]
        elif not is_list:
            obj.columns[column] = [obj.columns[column], value]
        elif is_list:
            obj.columns[column].append(obj.columns[column])
        else:
            raise TypeError('{} not supported (found at column: {!r} with value: {!r})'.format(
                type(obj), column, value
            ))
        return obj

    if hasattr(obj, 'columns'):
        if column in obj.columns:
            return _upsert()
    elif hasattr(obj, '__contains__'):
        if column in obj:
            return _upsert()
    elif not hasattr('__setattr__'):
        raise TypeError('{} not supported (found at column: {!r} with value: {!r})'.format(
            type(obj), column, value
        ))

    obj[column] = [value]

    return _upsert()

In [ ]:
def maybe_rename_column(column):
    suffix = 'right' if 'Right' in column else 'left'
    columns = 'ICDRSS', 'maculopathy'
    for col in columns:
        if column.endswith(col):
            return '_'.join((suffix, col))
        elif '{col} recorded'.format(col=col).lower() in column.lower():
            return '_'.join((suffix, col, 'photo'))
        else:
            pass
            #display('"{col} recorded" in "{column}" gave {r}'.format(col=col, column=column.lower(),
            #                                          r='{col} recorded'.format(col=col) in column.lower()))
    return column

In [ ]:
'''
elif np.array_equal(
    np.array(['Mod NPDR',
              'No DR',
              'Mild NPDR',
              'Treated PDR, Stable',
              'No gradable image', # np.nan,
              'Sev NPDR'], dtype=object),
    found_cols):

    return get_sum_df({
        'referrable': ['Mod NPDR', 'Mild NPDR', 'Treated PDR, Stable', 'Sev NPDR'],
        'non-referable': ['No DR'],
        'ungradable': ['No gradable image']
    })

# maculopathy
elif np.array_equal(
    np.array(['No diabetic maculopathy', #np.nan,
              'Maculopathy, unspecified'],
             dtype=object),
    found_cols):
    return get_sum_df({
        'referrable': ['Maculopathy, unspecified'],
        'non-referable': ['No diabetic maculopathy']
    })

elif np.array_equal(
    np.array(['No diabetic maculopathy', 'Maculopathy, unspecified',
              'No gradable image', 'HEx distant from fovea' # , np.nan
             ], dtype=object),
    found_cols):

    return get_sum_df({
        'referrable': ['Maculopathy, unspecified', 'HEx distant from fovea' ],
        'non-referable': ['No diabetic maculopathy'],
        'ungradable': ['No gradable image']
    })
    '''
pass

In [ ]:
'''
elif np.array_equal(
    np.array(['Mod NPDR', 'No DR', 'Mild NPDR', 'Treated PDR, Stable',
              'No gradable image', #np.nan,
              'Sev NPDR'], dtype=object),
    found_cols):            
    def ICDRSS_to_3_cat(column):
        if pd.isnull(column):
            return
        return {'Mod NPDR': 'referable',
                'No DR': 'non-referable',
                'Mild NPDR': 'referable',
                'Treated PDR, Stable': 'referable',
                'No gradable image': 'ungradable', #np.nan,
                'Sev NPDR': 'referable'}[column]

    return series.apply(ICDRSS_to_3_cat).to_frame()

# maculopathy
elif np.array_equal(
    np.array(['No diabetic maculopathy', #np.nan,
              'Maculopathy, unspecified'],
             dtype=object),
    found_cols):
    def maculopathy_to_3_cat(column):
        # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
        if pd.isnull(column):
            return
        return {'No diabetic maculopathy': 'non-referable',
                'Maculopathy, unspecified': 'non-referable'}[column]

    return series.apply(maculopathy_to_3_cat).to_frame()
elif np.array_equal(
    np.array(['No diabetic maculopathy', 'Maculopathy, unspecified',
              'No gradable image', 'HEx distant from fovea' # , np.nan
             ], dtype=object),
    found_cols):
    def maculopathy_to_3_cat(column):
        # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
        if pd.isnull(column):
            return
        return {'No diabetic maculopathy': 'non-referable',
                'Maculopathy, unspecified': 'referable',
                'No gradable image': 'ungradable',
                'HEx distant from fovea': 'referable'}[column]

    return series.apply(maculopathy_to_3_cat).to_frame()
'''
pass

In [ ]:
def to_3_cat(series):
    assert isinstance(series, pd.Series), 'Expected `pd.Series` got `{!r}`'.format(type(series))
    
    series_value_counts = pd.DataFrame(series) \
                            .reset_index() \
                            .rename(index={'Record ID': 'id'}) \
                            .drop('Record ID', axis=1) \
                            .apply(pd.value_counts)
    
    def get_sum_df(d):
        return pd.DataFrame.from_records([{k: series_value_counts.loc[v].sum().values[0]
                                           for k, v in d.items()}],
                                        index=[maybe_rename_column(series_value_counts.columns[0])])
    
    found_cols = (lambda a: a[~pd.isnull(a)])(series.unique())
    series.dropna(inplace=True)

    # ICDRSS
    if np.array_equal(
        np.array(['Treated PDR, Stable', # np.nan,
                  'No DR', 'Mild NPDR', 'Mod NPDR', 'Sev NPDR'], dtype=object),
        found_cols):
        
        
        def ICDRSS_to_3_cat(column): 
            if pd.isnull(column):
                return
            return {'Treated PDR, Stable': 'referable',
                    'No DR': 'non-referable',
                    'Mild NPDR': 'referable',
                    'Mod NPDR': 'referable',
                    'Sev NPDR': 'referable'}[column]
        
        return series.apply(ICDRSS_to_3_cat).to_frame()
    elif np.array_equal(
        np.array(['Mod NPDR', 'No DR', 'Mild NPDR', 'Treated PDR, Stable',
                  'No gradable image', #np.nan,
                  'Sev NPDR'], dtype=object),
        found_cols):            
        def ICDRSS_to_3_cat(column):
            if pd.isnull(column):
                return
            return {'Mod NPDR': 'referable',
                    'No DR': 'non-referable',
                    'Mild NPDR': 'referable',
                    'Treated PDR, Stable': 'referable',
                    'No gradable image': 'ungradable', #np.nan,
                    'Sev NPDR': 'referable'}[column]

        return series.apply(ICDRSS_to_3_cat).to_frame()

    # maculopathy
    elif np.array_equal(
        np.array(['No diabetic maculopathy', #np.nan,
                  'Maculopathy, unspecified'],
                 dtype=object),
        found_cols):
        def maculopathy_to_3_cat(column):
            # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
            if pd.isnull(column):
                return
            return {'No diabetic maculopathy': 'non-referable',
                    'Maculopathy, unspecified': 'non-referable'}[column]
        
        return series.apply(maculopathy_to_3_cat).to_frame()
    elif np.array_equal(
        np.array(['No diabetic maculopathy', 'Maculopathy, unspecified',
                  'No gradable image', 'HEx distant from fovea' # , np.nan
                 ], dtype=object),
        found_cols):
        def maculopathy_to_3_cat(column):
            # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
            if pd.isnull(column):
                return
            return {'No diabetic maculopathy': 'non-referable',
                    'Maculopathy, unspecified': 'referable',
                    'No gradable image': 'ungradable',
                    'HEx distant from fovea': 'referable'}[column]
        
        return series.apply(maculopathy_to_3_cat).to_frame()
    elif np.array_equal(
        np.array(['No diabetic maculopathy', 'Maculopathy, unspecified', 'No gradable image'], dtype=object),
        found_cols):
        def maculopathy_to_3_cat(column):
            # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
            if pd.isnull(column):
                return
            return {'No diabetic maculopathy': 'non-referable',
                    'Maculopathy, unspecified': 'referable',
                    'No gradable image': 'ungradable'}[column]
        
        return series.apply(maculopathy_to_3_cat).to_frame()        
    else:
        display(HTML('No match for {} DONE with {!r}'.format(
            series.to_frame()[:3].to_html(),
            series.unique()
        )))

In [ ]:
#selected_columns = filter(lambda column: 'hoto grading' in column or 'recorded' in column, df.columns)
#selected_columns = 'Ophthal photo grading: Right eye ICDRSS',
selected_columns = filter(
    lambda column: column.startswith('Photo grading') or column.startswith('Ophthal photo grading'),
    df.columns
)
cat3_df = pd.concat((to_3_cat(df[column].copy())
                     for column in selected_columns), sort=True)
cat3_df.apply(pd.value_counts)

In [ ]:
import operator
operator.getitem?
operator.getitem('foo', {'foo': 'bar'})

In [ ]:
def toFname(series):  # type: (pd.Series) -> pd.Series
    assert isinstance(series, pd.Series)
    
    axes2locationPid = {
      'Ophthal photo grading: Left eye ICDRSS': None,
      'Ophthal photo grading: Left eye maculopathy': None,
      'Ophthal photo grading: Right eye ICDRSS': None,
      'Ophthal photo grading: Right eye maculopathy': None,
      'Photo grading - diabetes: ICDRSS recorded for the left eye': None,
      'Photo grading - diabetes: ICDRSS recorded for the right eye': None,
      'Photo grading - diabetes: Maculopathy recorded for the left eye': None,
      'Photo grading - diabetes: Maculopathy recorded for the right eye': None
    }
    
    print(series)
    
    for key, value in series.iteritems():
        print('key: {!r}; value: {!r}'.format(key, value))
    
    def f(value):
        if pd.isnull(value): return value
        print('\nf::value: ', '{!r}'.format(value), 'of type', type(value), dir(value))
        pd.Series( axes2locationPid[value[0]] if not pd.isnull(value[1]) else value[1])
        return value

    return series.apply(f)
    

cat3_df[200:201].apply(toFname)

In [ ]:
LocationPid = namedtuple('LocationPid', ('location', 'pid', 'position'))

def parseFname(fname):
    bname = path.splitext(path.basename(fname))[0]
    buffer, location, pid, last_char = '', '', '', ''
    i = 0
    while i < len(bname):
        location = last_char
        if location + bname[i] in frozenset(('R1', 'R2', 'L1', 'L2')):
            pid = pid[:-1]
            break
        pid += bname[i]
        last_char = bname[i]
        i += 1
    return LocationPid(location=location, pid=pid, position='macula' if location[0] == 'R' else 'disc')

# R1 = Right macula     centred photo
# R2 = Right optic disc centred photo
# L1 = Left  macula     centred photo
# L2 = Left optic disc  centred photo

parseFname('DR SPOC Photo Dataset/6146/Upload/WA112325R2-8.jpg')

In [ ]:
def sql_gen(fname, diagnosis):
    location, pid = parseFname(fname)
    prefix = 'fundus_images/'
    quoted_location = urllib.parse.quote(
        '{prefix}DR SPOC Photo Dataset/6146/Upload/WA112325R2-4.jpg'.format(prefix=prefix),
        safe=''
    )
    return '\n'.join(map(lambda l: l.lstrip(' '), '''
        BEGIN TRANSACTION;

        INSERT INTO artifact_tbl (location, "contentType")
        VALUES ('{quoted_location}', 'image/jpeg');

        INSERT INTO categorise_tbl ("artifactLocation", "categoryEnumName", category, username)
        VALUES ('{quoted_location}',
                \t'Simple-categories', '{diagnosis}', 'spreadsheet');

        END TRANSACTION;
        '''.format(quoted_location=quoted_location, diagnosis=diagnosis).split('\n')))

print(sql_gen(#(
    'DR SPOC Photo Dataset/6146/Upload/WA112325R2-8.jpg', 'REFERABLE')
    #('DR SPOC Photo Dataset/6146/Upload/WA112325R2-9.jpg', 'NON-REFERABLE'),
    #('DR SPOC Photo Dataset/6146/Upload/WA112325R2-1.jpg', 'UNGRADABLE')
#))
)

In [ ]:
import psycopg2

result = urllib.parse.urlparse(environ['RDBMS_URI'])

with psycopg2.connect(
    database = result.path[1:],
    user = result.username,
    password = result.password,
    host = result.hostname
) as conn:
    with conn.cursor() as curs:
        curs.execute('SELECT 5*5')
        print(curs.fetchone())